In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine, correlation

%matplotlib inline

In [2]:
books = pd.read_csv( r'C:\Users\dheer\Desktop\Book Recommendation\Dataset_Books.csv', encoding='latin-1')
books.head(5)

,isbn,text_reviews_count,series,country_code,asin,is_ebook,average_rating,kindle_asin,publisher,num_pages,publication_day,isbn13,publication_month,publication_year,image_url,book_id,ratings_count,title,title_without_series,genre
0,0761458824,38,[],US,NaN,False,3.86,B008TWZWT0,Two Lions,40.0,1.0,9780761458821,2.0,2011.0,https://images.gr-assets.com/books/1355030251m...,10000006,89,Hatch!,Hatch!,Children
1,0761458050,102,[],US,NaN,False,3.92,B007VG9NHW,Two Lions,32.0,1.0,9780761458050,2.0,2011.0,https://images.gr-assets.com/books/1355028998m...,10000009,472,How to Teach a Slug to Read,How to Teach a Slug to Read,Children
2,0761458166,16,[],US,NaN,False,3.73,B008RDG6KA,Two Lions,40.0,1.0,9780761458166,2.0,2011.0,https://images.gr-assets.com/books/1355032487m...,10000014,52,Pond Walk,Pond Walk,Children
3,1400098777,2,['155138'],US,NaN,False,3.70,NaN,"Listening Library, Inc.",0.0,12.0,9781400098774,4.0,2005.0,https://s.gr-assets.com/assets/nophoto/book/11...,10000034,9,"Princess in Training (The Princess Diaries, #6)","Princess in Training (The Princess Diaries, #6)",Adult
4,1417730315,1,['155138'],US,NaN,False,3.70,B004I43MO0,Turtleback Books,269.0,27.0,9781417730315,5.0,2008.0,https://s.gr-assets.com/assets/nophoto/book/11...,10000045,3,"Princess in Training (The Princess Diaries, #6)","Princess in Training (The Princess Diaries, #6)",Adult


In [4]:
reviews = pd.read_csv( r'C:\Users\dheer\Desktop\Book Recommendation\Dataset_Reviews.csv', encoding='latin-1')

reviews.head(5)

,user_id,book_id,review_id,rating,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,c72d46b85ca8927108099a46f3767ac9,17162,0000001d043dd01e0093462f6bc522f2,4,Sat Oct 12 09:51:34 -0700 2013,Sat Oct 12 14:57:22 -0700 2013,Sat Oct 12 14:57:22 -0700 2013,Sat Oct 12 00:00:00 -0700 2013,0,0
1,d1f272951c6b327fd179dd005c7204f6,18592771,0000054dd36436c8c23268c76cccfcbe,5,Tue Feb 19 20:46:48 -0800 2013,Sat Dec 27 20:05:31 -0800 2014,Wed Oct 02 00:00:00 -0700 2013,Sun Sep 22 00:00:00 -0700 2013,4,0
2,08730e8f24ae2359935df82dee1df664,2318271,0000070d5b02a290c631438f4f618c48,5,Thu May 13 06:04:33 -0700 2010,Thu May 13 06:04:39 -0700 2010,NaN,NaN,0,0
3,d270f9e712238c5f864ecc28bdb87341,17314430,00000b455bb42173c5d057be1949c8e7,5,Sat Mar 16 17:54:26 -0700 2013,Sun Mar 24 11:35:48 -0700 2013,Sun Mar 24 11:35:48 -0700 2013,Sun Mar 24 00:00:00 -0700 2013,0,0
4,44a8f8f3f88d18dd851fd7c95fd0d1e1,594348,00000cb6ddb752160f004671e3a07510,4,Thu Aug 25 08:44:19 -0700 2016,Thu Sep 22 13:59:24 -0700 2016,Thu Sep 22 13:59:24 -0700 2016,NaN,0,0


In [4]:
books.shape

(1201024, 20)

In [5]:
reviews.shape

(10290666, 10)

In [6]:
# As the data is too big for computing, reducing the size by taking only the books which are rated more than 200 times and as we would be recommending the books with good average rating i.e. above 4.5
books_1 = books[books['ratings_count']>200]
books_1.shape

(226664, 20)

In [7]:
books_2 = books_1[books_1['average_rating']>=4.5]
books_2.shape

(3828, 20)

In [28]:
# Filtering by taking the books which are there in the books_2 table
reviews_new = reviews[reviews.book_id.isin(books_2.book_id)]
reviews_new.shape

(111288, 10)

In [9]:
# reviews_new.book_id.nunique()

3506

In [10]:
# reviews_new.user_id.nunique()

54018

In [11]:
# reviews_2 = reviews_new[reviews_new.rating >4.5 ]
# reviews_2.shape

(76493, 10)

In [12]:
reviews_2.book_id.nunique()

3319

In [13]:
reviews_2.user_id.nunique()

41888

In [14]:
# reviews_2.drop(reviews_2.columns[[2,4,5,6,7,8,9]], axis=1, inplace=True)

In [15]:
# reviews_2.shape

In [29]:
counts1 = reviews_2['user_id'].value_counts()
# counts1.head(5)

In [36]:
# Considering only those users who have rated at least 5 books as will be recommending only to those users
reviews_3 = reviews_new[reviews_new['user_id'].isin(counts1[counts1 >= 2].index)]
reviews_3.shape

(60949, 10)

In [88]:
# df = reviews_2.groupby('user_id').count()
# df

In [89]:
# df = df[df['book_id']>20]
# df

In [90]:
# df.reset_index(level = 0, inplace = True)

In [91]:
# reviews_3 = reviews_2[reviews_2.user_id.isin(df.user_id)]
# reviews_3.shape

In [37]:
print(reviews_3.user_id.nunique())
print(reviews_3.book_id.nunique())

13755
3151


In [71]:
ratings_matrix = reviews_3.pivot(index='user_id', columns='book_id', values='rating')
user_id = ratings_matrix.index
book_id = ratings_matrix.columns
print(ratings_matrix.shape)
ratings_matrix.head()

(13755, 3151)


book_id,5,397,866,868,869,871,873,1414,1421,2151,...,36317571,36325400,36337785,36344051,36389839,36398867,36409421,36411879,36413250,36504672
user_id,,,,,,,,,,,,,,,,,,,,,
0007f8dd09337afd986d765569cf0110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0008465bde1208c4cbe0d8f915d2eadf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0008931c0cde961e9c802c5a58196d23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001abeffb7caaf02c99963352f22a8fe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00238d8a4c276c47f5d5e242f54a8f28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:

n_users = ratings_matrix.shape[0] #considering only those users who gave explicit ratings
n_books = ratings_matrix.shape[1]
print(n_users, n_books)

13755 3151


In [74]:
final_ratings = ratings_matrix.fillna(ratings_matrix.mean(axis=0))
final_ratings

book_id,5,397,866,868,869,871,873,1414,1421,2151,...,36317571,36325400,36337785,36344051,36389839,36398867,36409421,36411879,36413250,36504672
user_id,,,,,,,,,,,,,,,,,,,,,
0007f8dd09337afd986d765569cf0110,4.845599,4.625,4.111111,4.327273,4.28125,4.5,4.287671,4.142857,5.0,5.0,...,5.0,0.0,5.0,5.0,3.833333,5.0,5.0,4.875,4.882353,0.0
0008465bde1208c4cbe0d8f915d2eadf,4.845599,4.625,4.111111,4.327273,4.28125,4.5,4.287671,4.142857,5.0,5.0,...,5.0,0.0,5.0,5.0,3.833333,5.0,5.0,4.875,4.882353,0.0
0008931c0cde961e9c802c5a58196d23,4.845599,4.625,4.111111,4.327273,4.28125,4.5,4.287671,4.142857,5.0,5.0,...,5.0,0.0,5.0,5.0,3.833333,5.0,5.0,4.875,4.882353,0.0
001abeffb7caaf02c99963352f22a8fe,4.845599,4.625,4.111111,4.327273,4.28125,4.5,4.287671,4.142857,5.0,5.0,...,5.0,0.0,5.0,5.0,3.833333,5.0,5.0,4.875,4.882353,0.0
00238d8a4c276c47f5d5e242f54a8f28,4.845599,4.625,4.111111,4.327273,4.28125,4.5,4.287671,4.142857,5.0,5.0,...,5.0,0.0,5.0,5.0,3.833333,5.0,5.0,4.875,4.882353,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff07bab21bcf970b61f5c7a642fade7,4.845599,4.625,4.111111,4.327273,4.28125,4.5,4.287671,4.142857,5.0,5.0,...,5.0,0.0,5.0,5.0,3.833333,5.0,5.0,4.875,4.882353,0.0
fff9a27567817b04a05b8244e8b5883d,4.845599,4.625,4.111111,4.327273,4.28125,4.5,4.287671,4.142857,5.0,5.0,...,5.0,0.0,5.0,5.0,3.833333,5.0,5.0,4.875,4.882353,0.0
fffc475c53c2c59e160a4274aec002cf,4.845599,4.625,4.111111,4.327273,4.28125,4.5,4.287671,4.142857,5.0,5.0,...,5.0,0.0,5.0,5.0,3.833333,5.0,5.0,4.875,4.882353,0.0


In [77]:
cosine = cosine_similarity(final_ratings)
np.fill_diagonal(cosine, 0 )
similarity_with_book = pd.DataFrame(cosine,index=final_ratings.index)
similarity_with_book.columns=final_ratings.index
similarity_with_book

user_id,0007f8dd09337afd986d765569cf0110,0008465bde1208c4cbe0d8f915d2eadf,0008931c0cde961e9c802c5a58196d23,001abeffb7caaf02c99963352f22a8fe,00238d8a4c276c47f5d5e242f54a8f28,0027e1049a2b4ddb347bea2ad7f73041,002eff40d3de8ff36174a48d26d93da7,0036e193ef216f74d150e31c5b2445bc,003c3cbe1f0bf247fc1bae43984e333b,004d5e96c8a318aeb006af50f8cc949c,...,ffe51b1da9d52dbdf133cb3628b4217b,ffe776933441cd373201fa7fbd5dd321,ffe7c93bb174f1a4990fe36c5b67141c,ffe85ffe7b49f673b859add8cf50ea0f,ffe883170a3d48f22a4bacf678c9d2bd,fff07bab21bcf970b61f5c7a642fade7,fff9a27567817b04a05b8244e8b5883d,fffc475c53c2c59e160a4274aec002cf,fffc9cfe8fd818f574c8c219b93274c0,ffff7cafdaf5196383cb2efca08fb6fe
user_id,,,,,,,,,,,,,,,,,,,,,
0007f8dd09337afd986d765569cf0110,0.000000,0.999895,0.999895,0.999895,0.999894,0.999894,0.999839,0.999893,0.999894,0.999782,...,0.999895,0.999895,0.999891,0.999894,0.999879,0.999896,0.999893,0.999894,0.999864,0.999868
0008465bde1208c4cbe0d8f915d2eadf,0.999895,0.000000,0.999999,0.999999,0.999998,0.999998,0.999944,0.999997,0.999998,0.999881,...,0.999999,0.999999,0.999995,0.999999,0.999983,0.999998,0.999997,0.999998,0.999968,0.999972
0008931c0cde961e9c802c5a58196d23,0.999895,0.999999,0.000000,0.999999,0.999998,0.999998,0.999944,0.999997,0.999998,0.999881,...,0.999999,0.999999,0.999995,0.999999,0.999983,0.999998,0.999997,0.999998,0.999968,0.999972
001abeffb7caaf02c99963352f22a8fe,0.999895,0.999999,0.999999,0.000000,0.999998,0.999998,0.999943,0.999997,0.999998,0.999881,...,0.999999,0.999999,0.999995,0.999998,0.999983,0.999998,0.999997,0.999998,0.999968,0.999972
00238d8a4c276c47f5d5e242f54a8f28,0.999894,0.999998,0.999998,0.999998,0.000000,0.999997,0.999942,0.999996,0.999998,0.999877,...,0.999998,0.999998,0.999994,0.999997,0.999983,0.999997,0.999996,0.999998,0.999967,0.999971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff07bab21bcf970b61f5c7a642fade7,0.999896,0.999998,0.999998,0.999998,0.999997,0.999997,0.999942,0.999996,0.999997,0.999880,...,0.999998,0.999998,0.999994,0.999998,0.999982,0.000000,0.999996,0.999997,0.999967,0.999971
fff9a27567817b04a05b8244e8b5883d,0.999893,0.999997,0.999997,0.999997,0.999996,0.999996,0.999941,0.999995,0.999996,0.999879,...,0.999997,0.999997,0.999993,0.999997,0.999981,0.999996,0.000000,0.999996,0.999966,0.999970
fffc475c53c2c59e160a4274aec002cf,0.999894,0.999998,0.999998,0.999998,0.999998,0.999997,0.999942,0.999996,1.000000,0.999870,...,0.999998,0.999998,0.999994,0.999998,0.999984,0.999997,0.999996,0.000000,0.999967,0.999971


In [78]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df



In [88]:
sim_user_30_m = find_n_neighbours(similarity_with_book,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
user_id,,,,,,,,,,,,,,,,,,,,,
0007f8dd09337afd986d765569cf0110,d61c8e1f3e8d39aad74ea385ce56b653,a1d0233964fc619f1e70ef952de42610,c92cade66da897a9ca18f15aac08ccfe,9c8cf13d48a6a64ffbfd7351c3a3c6b4,554b023fc79ceb3e8ab6da29e0e7f5b5,698aea6f16e23a5ebe0dab8c2acb3683,d8f651f88115ea135293f0efa761bdf9,ae560e2d0f86e4e88288fd5d50e61a44,5731bdd90cf4cac7eb7cd7c8b561893c,cf66cfe4716201890fbe985f962fd49e,...,33ae8fd1ecd2a9478c262456b9fbfc28,c17275ce0be2129e9129cc38e749a81e,e4ff54ee9082573273a299a679c408bc,57f7f1b20516553142ea7d0564492ac2,6911168129ec15c24326b47cc3f09a07,f44e6dcb268246023344e4825a0f9e07,fcb28276367a75560eb93f70a4062468,7d8cd914f2cba0ec9be8b9f227a8e589,745c33f028ac2c9c3703fcfb9d01baa3,311fd4f1416e4697d3513a3ec31fc3a4
0008465bde1208c4cbe0d8f915d2eadf,cfca2b6a31816c2dd77cf9128f6046b3,7260ce1d42b48b9efe1cf73c59d0c43d,e5f54d38ad4e42372ab1327f48c3d50f,acdb660e397aef59fdad12eed540671b,26757bb1551feb7d59bec29c1ff7de0c,68118d13a6e6f38f255435e455072e74,d891836169c4e89335d3859e7f839c98,54e08d519623871c5eb93f86c1048edc,1aead53c429cac53a718a3928e672604,892c38dc1cc6ac6675a2dca27f726af5,...,0e3b4af70efcc5dfbed7414f698c2379,be878b74ba81e34142bb74b9de093f8c,50820a30e5b23dbe5bf20eda2e643cc6,b8c9561ea68b8af8fb07adc40215bb45,5de2526b0c6370bd4f0335bb065c65fa,17eb1f1e107eeff5edba37ac5934bbe1,02e38e76ef355e6dc7d675c2970e6a56,0e3f724359e66b69eb411b3deebe9deb,b240a91bd6c36f9f4c1b324d0bb1eed4,ddb35fd0fbb0b71f2fa56d1e26633e8f
0008931c0cde961e9c802c5a58196d23,4428db6b5812e9fed46d514b89d9539d,655cd15edb97b620f0e7e76fcaf683da,3d1fbebbccd5f2e76acb8e21965573b3,b8eac413ab0811270c89d5e5d26ebdd9,82194d36319dc66815cd0c8f9f6fb126,86e89da20b45896ff9c16d2b2ad324c8,38e14cb031366c2e31570a6ab6e6adc4,767b006dd34e6ab6940f2901afc8b5f9,d74794110b1fe69e735094c00287f101,68dcd68e07330965133143589494ca38,...,6b6a601d35ff6d8ec4a0ddd5abd9de78,a8a0016d7b972be615e6eac5f90eb913,3fca421d4ad2fcad62629f185733d938,0b8a573c5691f7bee17253bfab39c4d7,1c4143a2a7f624eb3fde9255732eb1c6,8383287d97c9e64d31c74ca3f216667b,ce7ef42b58be2a472b06a5f160743d60,3aee3ffd8133a610763464a0860fa66e,470ecc3b73097d01bee2a4e4606fa9fd,14fb70445e7f660e566db258a6a6a3d9
001abeffb7caaf02c99963352f22a8fe,e300646dee2c374e507c3fb71610b2c7,bc49e03b72846b5e780c19816ec869fb,03e7587096eee5d2e6ac57fbd48daf42,815a79a86444ab8315806fbc3b14b3fc,be42b88d43b73094e1aa646d7a913b78,00b5f8855666ef10f0760c88677be57c,0666d3d556852da9317d64675a2a6fd5,8471d9e2f70a626fdc4143894b4c03d2,099d92ace460b516e5023ef992848080,d92e9a52cfe5d42325b70ec142af3f25,...,8f08531f67d2d09389d8f6f1b93fb0d9,b8e8ff0e3fc56bd2c67b8474af716013,beb707cae043b90d1e85c03c9cd5db7a,bec1ce51efee66c319bdaccd382bc511,52121738cc67b091882beba9c1fa404b,7f340aa2304a93a79a6cb864e70cb72e,748ecfc5d7e742163ee4ed691c4a1b70,143c6bf50a1973617e1d770ec1380447,7501a2a4795a9050b1df9727b84c51af,7521323dee70941a1eaf45b3bc7f3d96
00238d8a4c276c47f5d5e242f54a8f28,840db82f7426e3ff7a86cf945ed7719a,f07b83ebbbf7bee1b6a3051af3d6dc05,998a2914e96fcdf4ee73661b41d6b954,64e3e01af33ec6f0e0d22a929ab5fc9b,c745146ac279faf41f65222a902aee83,d66f0903756b7f5253a43695538ca072,8106870cbfa7fa90ebac9f5dc577e959,1d1bd9c21f4ba952c9a79b2dab017b1f,1295e88ceaef211132b1c1c256c99a42,5ae07a0d007629d51b3d7e4ba1dd28e4,...,4e0beeeaedc6eb2ae99c5299136f506a,a15488af34c3e63bfe26bfd32b8155c5,fe80d53272a0ff70e59b75eeab36a565,a979f0534b0298896fad0613f04a03e7,be553ed533c42a9ecad6da8e20038a61,e4ca8d8ebca2e0ca1b7c7b1f4c090bf9,efe6212792510abf832e945cf2e8b6b0,f3007d0f405cc9b5e2db2d1d4290f071,c02f9b2e050f2bc7398712efa1788544,37ad9d94266607647f15ff7178cd2f8d


In [82]:
Mean = reviews_3.groupby(by="user_id",as_index=False)['rating'].mean()

In [84]:
reviews_3 = reviews_3.astype({"book_id": str})
book_user = reviews_3.groupby(by = 'user_id')['book_id'].apply(lambda x:','.join(x))

In [92]:

def User_item_score1(user):
    Books_read_by_user = ratings_matrix.columns[ratings_matrix[ratings_matrix.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = book_user[book_user.index.isin(b)]
    l = ','.join(d.values)
    Books_read_by_similar_users = l.split(',')
    Books_under_consideration = list(set(Books_read_by_similar_users)-set(list(map(str, Books_read_by_user))))
    Books_under_consideration = list(map(int, Books_under_consideration))
    score = []
    for item in Books_under_consideration:
        c = final_ratings.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['user_id'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_book.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['agg_score','correlation']
        fin['score']=fin.apply(lambda x:x['agg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'book_id':Books_under_consideration,'score':score})
    top_10_recommendations = data.sort_values(by='score',ascending=False).head(10)
    Book_Name = top_10_recommendations.merge(books_2, how='inner', on='book_id')
    Book_Names = Book_Name.title.values.tolist()
    return Book_Names

In [94]:
user = str(input("Enter the user id to whom you want to recommend : "))
predicted_books = User_item_score1(user)
print(" ")
print("The Recommendations for User Id : ", user)
print("   ")
for i in predicted_books:
    print(i)
    
    



Enter the user id to whom you want to recommend : 0007f8dd09337afd986d765569cf0110
 
The Recommendations for User Id :  0007f8dd09337afd986d765569cf0110
   
Beneath the Stain - Part Seven
Skin (44 Chapters Spin-Off, #1)
Dirty Heart (Cole McGinnis, #6)
Dirty Heart (Cole McGinnis, #6)
Beneath the Stain - Part Five
Speed (44 Chapters Spin-Off, #2)
Lovely Trigger (Tristan & Danika, #3)
Something Like Autumn (Something Like, #3)
Brie Surrenders her Heart (After Graduation, #8)
The Saint (The Original Sinners, #5)
Long Way Down (Calloway Sisters, #4; Addicted, #3.2)
